In [2]:
import pandas as pd
import tweepy
import json
import requests
import os

# Gathering:

### WeRateDogs Twitter Archive:

In [2]:
archive = pd.read_csv('twitter-archive-enhanced.csv'); archive.head(1)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None


In [3]:
archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

### Tweet Image Predictions:

In [4]:
folder_name = 'tweet_image_predictions'

if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

response = requests.get(url)

In [5]:
with open(os.path.join(folder_name, url.split('/')[-1]), mode = 'wb') as file:
    file.write(response.content)

In [6]:
image_preds = pd.read_csv('./tweet_image_predictions/image-predictions.tsv', sep = '\t'); image_preds.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


### Twitter API:

In [7]:
# initialize the credentials of Twitter Developer Account
consumer_key = '7mBdsNPDYQWk4kgsWMMF3RnQH'
consumer_secret = 'eILB8OwpUKQc7xSb8CB9RfxezhdRHfGdwFcocSYFxsk2IkrbYo'
access_token = '1140993197037895680-VPJik4MsoXr8pTCpXCczdNpfJvV8hB'
access_secret = '37wszdnwEg3rIHt5I55GDNODN5kIFPOf6lGOVa6enQU32'

# authenticate as per tweepy docs
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

# create api object
api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

In [ ]:
# example:
tweet = api.get_status(892420643555336193)
tweet._json['favorite_count']

In [1]:
# make folder to store the twitter api data
folder_name = 'twitter_api_data'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# init counter to keep track of tweets collected and of the failed ids
i=1
failed_ids = []

# open file to write json objects from api
with open('./twitter_api_data/tweet.txt', 'w', encoding='utf8') as file:
    for _ in archive.tweet_id:
        
        # try-except block since few tweet IDs in the archive may have been deleted
        try: 
            tweet = api.get_status(_, tweet_mode = 'extended')
        except:
            failed_ids.append(_)
            i = i+1
            continue
        
        # print the number of tweets collected
        print(f'{_} | {i} of {len(archive.tweet_id)}')
        i = i+1
        
        # dump the json object corresponding to the tweet collected from the api
        json.dump(tweet._json, file, indent=4,  separators=(',', ': '))
        file.write('\n')

NameError: name 'os' is not defined